# Dependencies

In [ ]:
#INSTALLING DEPENDENCIES
import sys
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Data Loading 

In [ ]:
# Get CSV files list from a folder
data1 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_1_2021-01-01_-_2021-07-01_228.csv", sep=';')
data2 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2020_3086/Turbine_Data_Kelmarsh_1_2020-01-01_-_2021-01-01_228.csv", sep=';')
data3 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2019_3085/Turbine_Data_Kelmarsh_1_2019-01-01_-_2020-01-01_228.csv", sep=';')
data4 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2018_3084/Turbine_Data_Kelmarsh_1_2018-01-01_-_2019-01-01_228.csv", sep=';')

In [ ]:
plt.figure(figsize=(80, 48))
# Calculate the percentage of missing values in each column
missing_percentage = data.isnull().mean() * 100

# Create a barplot of the missing percentages using seaborn
sns.barplot(x=missing_percentage.index, y=missing_percentage, color='blue')
plt.xticks(rotation=90)  # Rotate the x-axis labels for readability
plt.ylabel('Percentage des valeurs manquantes')
plt.xlabel('Colonnes')
plt.title('Percentage des valeurs manquantes par Colonnes')
plt.show()

In [ ]:
data.tail()

In [ ]:
data_filtered.shape

## Data Cleaning 

In [ ]:
plt.figure(figsize=(20, 12))
# Calculate the percentage of missing values in each column
missing_percentage = data_filtered.isnull().mean() * 100

# Create a barplot of the missing percentages using seaborn
sns.barplot(x=missing_percentage.index, y=missing_percentage, color='blue')
plt.xticks(rotation=90)  # Rotate the x-axis labels for readability
plt.ylabel('Percentage des valeurs manquantes')
plt.xlabel('Colonnes')
plt.title('Percentage des valeurs manquantes par Colonnes')
plt.show()

In [ ]:
data_filtered.isnull().sum()

In [ ]:
print('Number of duplicated data' ,data_filtered.duplicated().sum())
data_filtered['# Date and time'] = pd.to_datetime(data_filtered['# Date and time'],format='mixed')
data_filtered = data_filtered.sort_values('# Date and time')
data_filtered.reset_index(drop=True, inplace=True)

In [ ]:
sns.relplot(x='Wind speed (m/s)', y='Power (kW)',data=data_filtered)

### Data Normalisation

In [ ]:
data = data_filtered
data=data.drop(['# Date and time'], axis=1)
data.tail()

In [ ]:
# Create a scatterplot with sns.relplot
sns.relplot(x='Wind speed (m/s)', y='Power (kW)', hue=anomaly_labels, data=data_filtered)

### Data Labelisation

In [ ]:
data_status21 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2021_3087/Status_Kelmarsh_1_2021-01-01_-_2021-07-01_228.csv", sep=';')
data_status21 = data_status21.drop(data_status21[data_status21['Duration'] == '-'].index)
data_status21.reset_index(drop=True, inplace=True)


data_status20 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2020_3086/Status_Kelmarsh_1_2020-01-01_-_2021-01-01_228.csv", sep=';')
data_status20 = data_status20.drop(data_status20[data_status20['Duration'] == '-'].index)
data_status20.reset_index(drop=True, inplace=True)


data_status19 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2019_3085/Status_Kelmarsh_1_2019-01-01_-_2020-01-01_228.csv", sep=';')
data_status19 = data_status19.drop(data_status19[data_status19['Duration'] == '-'].index)
data_status19.reset_index(drop=True, inplace=True)


data_status18 = pd.read_csv("C:/Users/dell/Dataset/Kelmarsh Dataset/Kelmarsh_SCADA_2018_3084/Status_Kelmarsh_1_2018-01-01_-_2019-01-01_228.csv", sep=';')
data_status18 = data_status18.drop(data_status18[data_status18['Duration'] == '-'].index)
data_status18.reset_index(drop=True, inplace=True)


tempdata1 = pd.concat([data_status18, data_status19])
tempdata2 = pd.concat([tempdata1, data_status20])
ds = pd.concat([tempdata2, data_status21])
#ds = data_status20
print('Number of incidents',ds.shape)

In [ ]:
# Convert fault start and end times to datetime objects
ds['Timestamp start'] = pd.to_datetime(ds['Timestamp start'],format='mixed')
ds['Timestamp end'] = pd.to_datetime(ds['Timestamp end'],format='mixed')

# Create a list of all the time periods during which the turbine was in a fault state
fault_periods = []
for _, row in ds.iterrows():
    start = row['Timestamp start']
    end = row['Timestamp end']
    fault_periods.append((start, end))

# Create a list of anomaly labels for each row of the dataset
anomaly_labels = []
for _, row in data_filtered.iterrows():
    timestamp = pd.to_datetime(row['# Date and time'],format='mixed')
    anomaly = 0
    for start, end in fault_periods:
        if start <= timestamp <= end:
            anomaly = 1
            break
    anomaly_labels.append(anomaly)

# Convert the anomaly labels list to a NumPy array
anomaly_labels = np.array(anomaly_labels)



In [ ]:
print('Number of anomalies in the dataset :',anomaly_labels.sum())

### Saving the data 

In [ ]:
# file for data from 2018-2021 : Kelmarsh_turbine1(2018-2021).csv
# file for 2020 data Kelmarsh_turbine1-2020.csv

turbine = data_scaled
turbine['anomaly'] = anomaly_labels
turbine.to_csv('Kelmarsh_turbine1(2018-2021).csv')